# Create a table with TCGA data

In [ ]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,numpy,matplotlib,regex -a Filippo_Valle -g -r -b -w

In [ ]:
# import libraries
import os
import sys
import pandas as pd
import numpy as np
import regex as re
from matplotlib import pyplot as plt
import time

Set *working_dir* to the directory where you downloaded files in *data*

In [ ]:
working_dir = "/home/jovyan/work/phd/miRNA"
os.chdir(working_dir)

Prepare a list of genes that satisfied filters described by *Dey et al.* [Visualizing the structure of RNA-seq expression data using grade of membership models](https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1006599)

In [ ]:
df_genes = pd.read_csv("https://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None).set_index(0)
select_genes = df_genes.index.values

In [ ]:
gene_type = "gene" # "mirna" or "gene"

In [ ]:
if gene_type == "mirna":
    df = pd.DataFrame()
    dirs = os.listdir("data_mirna/")
else:
    df = df_genes.copy()
    dirs = os.listdir("data/")

In [ ]:
len(dirs)

In [ ]:
len(df)

In [ ]:
if gene_type == "mirna":
    ##miRNA
    def getFilenameFromDir(directory):
        for element in os.listdir(directory):
            if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.mirbase21\.mirnas\.quantification][\.txt]{0,4}[\.gz]{0,3}",element):
                cfile = element
                print(element)
                return cfile
else:
    #fpkm
    def getFilenameFromDir(directory):
        if ".DS_Store" in directory:
            return None
        for element in os.listdir(directory):
            if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.FPKM]{5}.txt[\.gz]{0,3}",element):
                cfile = element
                print(element)
                return cfile
        raise BaseException("Not found %s"%os.listdir(directory))

In [ ]:
maxacceptables = 15000
added = len(df.columns)
for i,cdirectory in enumerate(dirs):
    if re.match("manifest\.txt",cdirectory):
        print("SKIPPING %s "%cdirectory)
        continue
    if "Icon" in cdirectory:
        print("SKIPPING %s "%cdirectory)
        continue
    if ".DS_Store" in cdirectory:
        print("SKIPPING %s "%cdirectory)
        continue
    if gene_type == "mirna":
        ## mi RNA
        cfile = getFilenameFromDir("data_mirna/%s"%cdirectory) # miRNA only!
        cdf = pd.read_csv(("%s/data_mirna/%s/%s"%(working_dir,cdirectory,cfile)), sep='\t') # miRNA only!
        cdf.drop(labels=["reads_per_million_miRNA_mapped","cross-mapped"], axis=1, inplace=True) #miRNA only
        cdf.set_index("miRNA_ID", inplace=True)
        cdf.rename(columns={"read_count": cfile}, inplace=True)
        old_L = len(df.columns)
        df = df.join(cdf, how="outer")    
    else:
        ## FPKM
        cfile = getFilenameFromDir("data/%s"%cdirectory)
        cdf = pd.read_csv(("data/%s/%s"%(cdirectory,cfile)), sep='\t', header=None)
        cdf.columns = ["gene", cfile[:]]
        cdf['gene'] = [gene[:15] for gene in cdf['gene']]
        cdf.set_index('gene',inplace=True)
        cdf = cdf.reindex(index=df_genes.index)
        old_L = len(df.columns)
        df = df.join(cdf, how="outer")
    
    ## common
    if len(df.columns) != old_L+1:
        print(*sys.exc_info())
        raise(Exception("Not able to add: %s"%cfile))
    if added >= maxacceptables:
        break
print(added, i)

In [ ]:
#df = pd.read_csv(("%s/mainTable.csv"%working_dir))

In [ ]:
print(("genes:%d\trealizations:%d"%(len(df.index),len(df.columns))))

In [ ]:
if gene_type == "mirna":
    df.dropna(how='all', axis=0).round(decimals=2).to_csv("mainTable_mirna.csv", index=True)
else:
    df_files = pd.read_csv("files_fpkm.dat").set_index("file_name").reindex(index=df.columns)
    df.columns = df_files["cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id"]
    df.dropna(how='all', axis=0).round(decimals=2).to_csv("mainTable_fpkm.csv", index=True)